# 📌 자막 데이터 크롤링  
* urllib 라이브러리 사용해 파일 다운로드

### ✏️ 애니메이션 자막 페이지 목록 받아오기

In [ ]:
# 라이브러리 읽어들이기
import requests
import re
import os
from bs4 import BeautifulSoup as bs

# 파일 주소 크롤링
def bs_parsing(url):
  page_url = url
  response = requests.get(root_url + page_url)
  
  soup = bs(response.text, "html.parser")
  elements = soup.select("table#flisttable td > a")
    
  page_list = []
  
  # 상세 페이지 주소 리스트화
  for (idx, el) in enumerate(elements, 1):
    page_list.append(root_url + el.attrs["href"].replace("%2F", "/"))
    
  return page_list

def div_list(page_list):
  index_list = ["spec"]
  for al in range(65, 91):
    index_list.append(chr(al))

def save_text_file(page_list):
  f = open("C:/Users/sodud/study/ssafy_spec_pjt/subtitle/page_url_list.txt", "w", encoding="UTF8")
  for page in page_list:
    f.write(page)
    f.write("\n")
  f.close

# 자막 사이트
root_url = "https://kitsunekko.net"

# 일본어 자막 목록 페이지
page_url = "/dirlist.php?dir=subtitles/japanese/"

page_list = bs_parsing(page_url)
save_text_file(page_list)
print(page_list)
# index_list = div_list(page_list)

In [10]:
import requests
import re
import os
from bs4 import BeautifulSoup as bs
from inspect import getfile
from urllib import request
from urllib.error import HTTPError

def read_text_file(page_list_file, start, unit):
  
  flag = True
  f = open(page_list_file, "r", encoding="UTF8")
  lines = f.readlines()[start: start+unit]
  
  if len(lines) == 0:
    return
  
  if len(lines) < unit:
    flag = False
  
  for line in lines:
    if line[-3:] != "%2F":
      pass
    response = requests.get(line[:-1])
    soup = bs(response.text, "html.parser")
    files = soup.select("table#flisttable td > a")
    
    file_urls = []
    
    # 파일 읽기
    for f in files:
      file_path= f.attrs["href"]
      
      # srt 파일만 저장
      if file_path[-3:] == "srt":
        file_urls.append("/" + file_path)
      else:
        pass
    
    print(file_path)
    file_search(file_urls)
    
  if flag == False:
    return
  f.close
  return read_text_file(page_list_file, start+unit, unit)

# 파일이 존재하는지 확인
def file_search(file_urls):
  for file_url in file_urls:
    # 파일 이름 추출
    depth = file_url.split("/")
    filename = depth[len(depth)-1]
    try:
      path = "C:\\Users\\sodud\\study\\ssafy_spec_pjt\\subtitle\\data\\"
      if os.path.isfile(path + filename):
        print("이미 존재하는 파일: " + filename)
      else:
        get_download(root_url + file_url.replace(" ", "%20").replace("'", "%27"), filename, path)
        
    except HTTPError as err:
      print("다운로드 중 에러 발생: " + filename + "/" + err)
      pass

# 파일 다운로드
def get_download(url, fname, dir):
  try:
    os.chdir(dir)
    request.urlretrieve(url, fname)
    print("다운로드 완료: " + fname)
  except HTTPError as err:
    print("에러 발생:" + err)
    return

# 자막 사이트
root_url = "https://kitsunekko.net"

# 자막 파일 페이지 목록 파일
page_list_file = "C:/Users/sodud/study/ssafy_spec_pjt/subtitle/page_url_list.txt"

read_text_file(page_list_file, 0, 10)

subtitles/japanese/Darwin's Game/[HorribleSubs] Darwin's Game S1 [1080p].zip


TypeError: can only concatenate str (not "HTTPError") to str

### ✏️ 자막에서 텍스트만 추출

In [ ]:
import os
import re

# 자막 텍스트만 추출
def get_subtitle(file_name):
  print("# 파일명:", file_name)
  sub_text = []
  
  # 파일 열기
  with open(path + file_name, "rt", encoding="UTF8") as sub_file:
    print(sub_file)
    
    # 정규식 정의
    regex = re.compile(r'\d*\n')
    time = re.compile(r'(\d{2}:\d{2}:\d{2})')
    temp = re.compile(r"'\n'|' '")
    ch_list = ["{\\an8}", "\n", "\u3000", "♪～～♪", "1", "♬～"]
    
    # 파일 라인별로 텍스트만 추출
    for line in sub_file:
      if not (regex.match(line) or time.search(line) or temp.match(line) or len(line)==0):
        line = line.strip()
        for ch in ch_list:
          line = line.replace(ch, "")
        sub_text.append(line)
    
  with open(path + file_name, 'w', encoding="UTF8") as f:
    f.writelines(sub_text)
    f.wirte("\n")
  
  return sub_text

path = "C:\\Users\\sodud\\study\\ssafy_spec_pjt\\subtitle\\data\\"

file_list = os.listdir(path)
sub_files = [sub_file for sub_file in file_list]

for sub_file in sub_files:
  get_subtitle(sub_file)
  print(sub_file + " 수정 완료")